# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
#!zip -r /home/jupyter/file.zip /home/jupyter/*

In [ ]:
#!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Train
Train on custom data

In [ ]:
# Weights & Biases (optional)
%pip install -q wandb  

In [12]:
!pip install wandb --upgrade

     |████████████████████████████████| 2.0 MB 4.4 MB/s eta 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.10.12
    Uninstalling wandb-0.10.12:
      Successfully uninstalled wandb-0.10.12


In [ ]:
#import os
#WANDB_API_KEY= 'b9ce0ac37518ed3979101e8a8eeaf27fc3a13139'
#os.environ['WANDB_API_KEY'] = WANDB_API_KEY 
#API key Giulia

In [6]:
import os
WANDB_API_KEY= '9c9dc0309a6183861727edc3c97cc9ae781503bb' 
os.environ['WANDB_API_KEY'] = WANDB_API_KEY
#API key Kutay

In [7]:
!wandb login

wandb: Currently logged in as: kutay (use `wandb login --relogin` to force relogin)


In [ ]:
# Tensorboard (optional)
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
#!unzip st_modified.zip

In [10]:
!pwd

/home/jupyter/yolov5


In [2]:
%cd yolov5
!ls

/home/jupyter/yolov5
Dockerfile   detect.py	      requirements.txt		   tutorial.ipynb
LICENSE      hal_dataset.zip  runs			   utils
README.md    hubconf.py       storage_tank_data_augmented  wandb
__pycache__  models	      storage_tank_new		   weights
data	     n_aug_st	      test.py
dataset.zip  overhead_cranes  train.py


In [ ]:
#!pip install torchvision==0.8.2

In [ ]:
  ### pytorch imbalanced dataset sampler installation  
#!git clone https://github.com/ufoym/imbalanced-dataset-sampler.git
#%cd imbalanced-dataset-sampler 
#!python setup.py install
#clonned  https://github.com/ufoym/imbalanced-dataset-sampler for dataset balancing7
#%cd ..

# Transfer learning with frozen layers

set freez to 'false' if you do not want to freeze

# Train


In [18]:
yolo_version = 's'
#epochs = 250
epochs = 100
batches= 64
#batches= 16
#yaml= 'overhead_cranes/data.yaml'
#project = 'runs/train/overhead_cranes'
#model = 'models/yolov5'+yolo_version+'.yaml'

#yaml= 'storage_tank_new/data.yaml'
#project = 'runs/train/storage_tank_new'
#model = 'models/yolov5'+yolo_version+'.yaml'



yaml= 'full_dataset/data.yaml'
project = 'runs/train/full_dataset'
model = 'models/yolov5'+yolo_version+'.yaml'

which_weights = 'coco' # coco, cust or our best
freez = 'false'
#img_size = 416
img_size = 640

if freez == 'true':
    freez_name = 'frz'
elif freez == 'false':
    freez_name = 'not_frz'

#freeze = ['model.%s.' % x for x in range(10)]

if which_weights == 'st':
    weights='runs/train/st_modified/250e_16b_s_not_frz_coco_640img/weights/last.pt' # our best weights must be moved here
elif which_weights == 'coco':
    weights = 'weights/yolov5'+yolo_version+'.pt' # yolov5s.pt
elif which_weights == 'cust':
    weights = 'runs/train/st_modified/'+str(epochs)+'e_'+str(batches)+'b_'+yolo_version+'_'+str(img_size)+'img/weights/last.pt'
    #weights = 'runs/train/storage_tank_2/'+str(epochs)+'e_'+str(batches)+'b_'+yolo_version+'/weights/last.pt'

name=str(epochs)+'e_'+str(batches)+'b_'+yolo_version+'_'+freez_name+'_'+which_weights+'_'+str(img_size)+'img'

# include  --evolve in the command if you want to do hyperparameters evolution --> takes hundreds of hours
#original train command


!python train.py --img-size {img_size} --batch {batches} --epochs {epochs} --data {yaml} --cfg {model} --weights {weights} --nosave --cache --project {project} --freezing {freez} --name {name}
#!python train.py --img-size {img_size} --batch {batches} --epochs {epochs} --data {yaml} --cfg {model} --weights '' --nosave --cache --project {project} --freezing {freez} --name {name}
#!python train.py --img-size {img_size} --batch {batches} --epochs {epochs} --data {yaml} --cfg {model} --weights {weights} --nosave --project {project} --freezing {freez} --name {name}

Your branch is behind 'origin/master' by 135 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Using torch 1.7.1 CUDA:0 (Tesla T4, 15109.75MB)


Namespace(adam=False, batch_size=64, bucket='', cache_images=True, cfg='models/yolov5s.yaml', data='full_dataset/data.yaml', device='', epochs=100, evolve=False, exist_ok=False, freezing='false', global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='100e_64b_s_not_frz_coco_640img', noautoanchor=False, nosave=True, notest=False, project='runs/train/full_dataset', quad=False, rect=False, resume=False, save_dir='runs/train/full_dataset/100e_64b_s_not_frz_coco_640img12', single_cls=False, sync_bn=False, total_batch_size=64, weights='weights/yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train/full_dataset", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, '

## Local Logging

All results are logged by default to `runs/train/exp_i`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, etc. View train and test jpgs to see mosaics, labels, predictions and augmentation effects. Note a **Mosaic Dataloader** is used for training (shown below), a new concept developed by Ultralytics and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
Image(filename='runs/train/exp/train_batch0.jpg', width=800)  # train batch 0 mosaics and labels


Training losses and performance metrics are also logged to [Tensorboard](https://www.tensorflow.org/tensorboard) and a custom `results.txt` logfile which is plotted as `results.png` (below) after training completes. Here we show YOLOv5s trained on COCO128 to 300 epochs, starting from scratch (blue), and from pretrained `--weights yolov5s.pt` (orange).

In [ ]:
from utils.plots import plot_results 
plot_results(save_dir='runs/train/exp')  # plot all results*.txt as results.png
Image(filename='runs/train/exp/results.png', width=800)

# Test

In [ ]:
# Run 
epochs = 250
#batches= 16
batches= 8
yolo_version = 'l'
name=str(epochs)+'e_'+str(batches)+'b_'+yolo_version

#epochs_and_batches=name
weights= 'runs/train/storage_tank_2_x/'+name+'/weights/last.pt'
data = 'storage_tank_new/data.yaml'
save_test_path = 'runs/test/storage_tank_2/'
fold_name=name+''
!python test.py --weights {weights} --data {data} --img 640 --iou 0.65 --project {save_test_path} --name {fold_name}

## Show results of testing

In [ ]:
from IPython.display import Image

path='runs/storage_tank_new/test/300e_64b'
#Image(filename=path+'/precision_recall_curve.png', width=800)  # train batch 0 mosaics and labels
#Image(filename=path+'/confusion_matrix.png', width=800)  # train batch 0 mosaics and labels
#Image(filename=path+'/test_batch0_labels.jpg', width=800)  # train batch 0 mosaics and labels
#Image(filename=path+'/test_batch0_pred.jpg', width=800)  # train batch 0 mosaics and labels
#Image(filename=path+'/test_batch1_labels.jpg', width=800)  # train batch 0 mosaics and labels
Image(filename=path+'/test_batch1_pred.jpg', width=800)  # train batch 0 mosaics and labels


# 3. New inference

`detect.py` runs inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases).

In [22]:
!ls

079_0056_cut_10.MOV	      imbalanced-dataset-sampler  yolov5
YOLOv5_Tutorial.ipynb	      tutorials
custom_training_yolov5.ipynb  yolov4_custom.ipynb


In [3]:
%cd yolov5

/home/jupyter/yolov5


In [4]:
!ls

Dockerfile   detect.py	      requirements.txt		   train.py
LICENSE      detect_my.py     runs			   tutorial.ipynb
README.md    full_dataset     storage_tank_data_augmented  utils
__pycache__  hubconf.py       storage_tank_new		   utils_obj
data	     models	      test.py			   wandb
dataset.zip  overhead_cranes  test.txt			   weights


In [ ]:
# epochs_and_batches=str(epochs)+'e_'+str(batches)+'b'
# weights= 'runs/train/storage_tank_2/train/'+epochs_and_batches+'/weights/last.pt'
# data_to_infer = 'storage_tank_new/synthetic_data'
# confidence = 0.25
# project_folder = 'runs/detect/synthetic_detected/'
# !python detect.py --weights {weights} --img 640 --conf 0.25 --source {data_to_infer} --project {project_folder} --name {epochs_and_batches}

In [ ]:
Image('runs/detect/synthetic_detected/1.JPG')

In [1]:
source = '../079_0056_cut_10.MOV'
# source = './HR1/'
weight= 'runs/train/full_dataset/100e_64b_s_not_frz_coco_640img/weights/last.pt'
# weight = 'weights/last (1).pt'

In [32]:
!python detect.py --source {source} --weights {weight} --conf 0.6 --img-size 640

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.6, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=True, save_txt=False, source='../079_0056_cut_10.MOV', update=False, view_img=False, weights=['runs/train/full_dataset/100e_64b_s_not_frz_coco_640img/weights/last.pt'])
Using torch 1.7.1 CUDA:0 (Tesla T4, 15109.75MB)


Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients, 16.8 GFLOPS
video 1/1 (1/328) /home/jupyter/079_0056_cut_10.MOV: 384x640 3 nozzles, Done. (0.020s)
video 1/1 (2/328) /home/jupyter/079_0056_cut_10.MOV: 384x640 3 nozzles, Done. (0.014s)
video 1/1 (3/328) /home/jupyter/079_0056_cut_10.MOV: 384x640 2 nozzles, Done. (0.014s)
video 1/1 (4/328) /home/jupyter/079_0056_cut_10.MOV: 384x640 4 nozzles, Done. (0.014s)
video 1/1 (5/328) /home/jupyter/079_0056_cut_10.MOV: 384x640 4 nozzles, Done. (0.014s)
video 1/1 (6/328) /home/jupyter/079_0056_cut_10.MOV: 384x640 4 nozzles, Done. (

In [ ]:
!pwd

In [ ]:
%cd yolov5

# Download video from google storage

In [1]:
!pip install google

     |████████████████████████████████| 45 kB 959 kB/s eta 0:00:011


In [29]:
from google.cloud import storage
import urllib.request as req
import datetime
import google.auth

credentials, project = google.auth.default()


In [30]:
client = storage.Client()
bucket = client.get_bucket('ddproc_videos')

fname = '079_0056.MOV'
blob = bucket.get_blob(fname)
# print(blob)


blob.download_to_filename(fname) # download file to the machine
# cap = cv2.VideoCapture('079_0056_cut_10.MOV')


In [ ]:
%pip install tensorflow

# Run detection +  img sim +  tracking

In [32]:
!python detect_my.py --start_frame 'begin' --end_frame 'end' --source '079_0056.MOV'

*****************************************************************************
last.pt
*****************************************************************************
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.6, device='', end_frame=['end'], exist_ok=False, img_size=416, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='079_0056.MOV', start_frame=['begin'], update=False, view_img=False, weights='runs/train/n_aug_st/100e_64b_s_not_frz_coco_640img3/weights/last.pt', yaml_file='../storage_tank.yaml')
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x56307f35d700] Referenced QT chapter track not found
Using torch 1.7.1 CUDA:0 (Tesla T4, 15109.75MB)


Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients, 16.8 GFLOPS
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5630f1dd2c00] Referenced QT chapter track not found
detection... |                                | 0/10306video 1/1 (1/10306) /home/jupyter/yolov5/079_0056.MOV: 1
1
detection...